In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
sns.set()
Candledf=pd.read_csv('../result.dat',low_memory=False)
Candledf['ndatetime']=pd.to_datetime(Candledf['ndatetime'],format='%Y-%m-%d %H:%M:%S.%f')
print(Candledf.tail())

                ndatetime   open   high    low  close  volume
10225 2021-11-17 10:15:37  17707  17742  17701  17742   12000
10226 2021-11-17 11:02:35  17742  17759  17733  17753   12000
10227 2021-11-17 11:50:34  17753  17761  17719  17740   12000
10228 2021-11-17 12:54:48  17740  17742  17710  17722   12000
10229 2021-11-17 13:27:00  17722  17723  17722  17722     976


In [2]:
(Candledf.close.shift(-20) / Candledf.close > 1).astype(int).tail()

10225    0
10226    0
10227    0
10228    0
10229    0
Name: close, dtype: int32

In [3]:
def triple_barrier(price, ub, lb, max_period):
    
    def end_price(s):
        return np.append(s[(s / s[0] > ub) | (s / s[0] < lb)], s[-1])[0]/s[0]
    
    r = np.array(range(max_period))
    
    def end_time(s):
        return np.append(r[(s / s[0] > ub) | (s / s[0] < lb)], max_period-1)[0]

    p = price.rolling(max_period).apply(end_price, raw=True).shift(-max_period+1)
    t = price.rolling(max_period).apply(end_time, raw=True).shift(-max_period+1)
    t = pd.Series([t.index[int(k+i)] if not math.isnan(k+i) else np.datetime64('NaT') 
                   for i, k in enumerate(t)], index=t.index).dropna()

    signal = pd.Series(0, p.index)
    signal.loc[p > ub] = 1
    signal.loc[p < lb] = -1
    ret = pd.DataFrame({'triple_barrier_profit':p, 'triple_barrier_sell_time':t, 'triple_barrier_signal':signal})

    return ret

In [12]:
ret = triple_barrier(Candledf.close, 1.07, 0.97, 20)


In [16]:
ret[ret.triple_barrier_signal < 0].count()

triple_barrier_profit       308
triple_barrier_sell_time    308
triple_barrier_signal       308
dtype: int64